<h1>Matt's eCFR API Demonstrator</h1>
<p>An intro to Jupyter Notebooks</p>
<p><i>Teach yourself Python using the eCFR API</i></p>

In [1]:
# Globals
# Do my imports - datetime for date operations, pandas for data analysis, aiohttp asyncio requests for HTTPS REST API requests, json for, well, javascript object notation, duh
import datetime as dt
import pandas as pd
import aiohttp as aio
import asyncio as asy   
import requests as rq
import json as json

# Set my constants
base_url = "https://www.ecfr.gov/api/"
titles_url = base_url + "versioner/v1/titles.json"
agencies_url = base_url + "admin/v1/agencies.json"
title_and_chapter_versions_url = base_url + "versioner/v1/versions/title-"
data_path = "../data/"

In [2]:
# Define my eCFR API wrapper class
class eCFR_API_Wrapper:
    """
    A wrapper for the eCFR API to fetch and process data.
    """

    def __init__(self, base_url):
        self.base_url = base_url

    def fetch_data(endpoint):
        """
        Fetch data from the eCFR API.
        """
        try:
            response = rq.get(f"{endpoint}")
            if response.status_code == 200:
                return response.json()
            else:
                response.raise_for_status()
        except: 
            print(f"Error fetching data from {endpoint}")
            return None
        
    def fetch_data_asynch(endpoint):    
        """
        Fetch data from the eCFR API asynchronously.
        """
        async def fetch(session, url):
            async with session.get(url) as response:
                return await response.json()

        async def main():
            async with aio.ClientSession() as session:
                return await fetch(session, endpoint)

        loop = asy.get_event_loop()
        return loop.run_until_complete(main())

In [3]:
class eCFR_HouseKeeping:

    titles_json = None
    agencies_json = None
    slugs = []
    titles_and_chapters = []


    def __init__(self, data_path):
        self.data_path = data_path

    def fetch_titles_data(self):
        """
        Fetch the titles data from the eCFR API.
        """
        try:
            self.titles_json = eCFR_API_Wrapper.fetch_data(titles_url)
            print("Fetching titles data from the API...")
            return self.titles_json
        except Exception as e:
            print(f"Error fetching titles data: {e}")
            return None 
        
    def fetch_agencies_data(self):
        """
        Fetch the agencies data from the eCFR API.
        """
        try:
            self.agencies_json = eCFR_API_Wrapper.fetch_data(agencies_url)
            print("Fetching agency data from the API...")
            return self.agencies_json
        except Exception as e:
            print(f"Error fetching agencies data: {e}")
            return None    
        
        
    def refresh_agencies(self):
        """
        Fetch the agencies data from the eCFR API.
        """
        # fetch the agencies data from the API
        self.fetch_agencies_data()
        # if we retrieved agencies_json, write it to the file
        if self.agencies_json:
            try:
                with open(self.data_path + 'agencies.json', 'w') as myagenciesfile:
                    json.dump(self.agencies_json, myagenciesfile)
                return "Agency data available, data fetched and written to agencies.json"
            except IOError as e:
                return (f"Error writing to agencies.json: {e}")
        else:
            return "Unable to fetch agencies data from the API"

    def load_agency_data(self):
        """
        Load the agency data from the agencies.json file.
        """
        try:
            with open(self.data_path + 'agencies.json', 'r') as myagenciesfile:
                self.agencies_json = json.load(myagenciesfile)
                return self.agencies_json
        except FileNotFoundError:
            print("No agencies.json found, need to refresh")
            self.refresh_agencies()
            return self.agencies.json

    def refresh_titles(self):
        """
        Fetch the titles data from the eCFR API.
        """
        # first, see if the titles.json file exists and is fresh    
        try:
            with open(self.data_path + 'titles.json', 'r') as mytitlesfile:
                self.titles_json = json.load(mytitlesfile)
        except FileNotFoundError:
            self.titles_json = None 
        if(self.titles_json is not None):
            # Get the current date and calculate the date 30 days ago
            thirtydaysback = dt.datetime.now() - dt.timedelta(days=30)
            lastbestdate = None
            #attempt to get data from the titles.json file
            if(self.titles_json.get('titles', [])):
                # check the up_to_date_as_of field and populate lastbestdate
                titles_dict = self.titles_json.get('titles', [])
                key_to_find = "up_to_date_as_of"
                up_to_date_values = [t[key_to_find] for t in titles_dict if key_to_find in t]
                lastbestdate = max(list(filter(None, up_to_date_values)))       
            if lastbestdate > str(thirtydaysback):
                print(f" with good data as of {lastbestdate}")
                # we have fresh data, so we can use it as is
            else:
                # we don't have fresh data, so fetch it from the API
                self.fetch_titles_data()
        else:
            # fetch the titles data from the API
            self.fetch_titles_data()
  
        #we have titles_json, write it to the file
        try:
            with open(self.data_path + 'titles.json', 'w') as mytitlesfile:
                json.dump(self.titles_json , mytitlesfile)
            return "Titles data available, data fetched and written to titles.json"
        except IOError as e:
            return (f"Error writing to agencies.json: {e}")

    def load_title_data(self):
        """
        Load the title data from the titles.json file.
        """
        try:
            with open(self.data_path + 'titles.json', 'r') as mytitlesfile:
                self.titles_json = json.load(mytitlesfile)
                return self.titles_json
        except FileNotFoundError:
            print("No agencies.json found, need to refresh")
            self.refresh_titles
            return self.titles_json

    def refresh_title_and_chapter_versions(self, title, chapter):    
        """
        Refresh the title and chapter version data.
        """
        my_title_url = title_and_chapter_versions_url + title + ".json?chapter=" + chapter 
        # fetch the title and chapter version data from the API
        try:
            title_and_chapter_version_json = eCFR_API_Wrapper.fetch_data(my_title_url)
        except Exception as e:
            return (f"Error fetching title version data: {e}")
        # if we have version data, write it to the file
        if title_and_chapter_version_json:
            myfile = "title"+title+"chapter"+chapter+"_versions.json"
            try:
                with open(self.data_path + 'versions/'+myfile, 'w') as myversionfile:
                    json.dump(title_and_chapter_version_json, myversionfile)
                return "Version data available, data fetched and written to "+myfile
            except IOError as e:
                return (f"Error writing to versions.json: {e}")
        else:
            return "Unable to fetch versions data from the API"

    def load_title_and_chapter_versions(self, title,chapter):
        """
        Load the title and chapter versions from an individual file.
        """
        try:
            with open(self.data_path + 'versions/title'+str(title)+'chapter'+str(chapter)+"_versions.json", 'r') as mytitlesfile:
                title_and_chapter_versions = json.load(mytitlesfile)
                return title_and_chapter_versions
        except FileNotFoundError:
            print("No title"+title+" chapter"+chapter+" versions found, need to fetch data from API")
            return None

    def refresh_all_versions_byslug(self):
        """
        Fetch all - call refresh_title_and_chapter_versions for all agency slugs
        """
        self.refresh_agencies()  # Ensure we have the latest agencies data
        # blank the slug array
        self.slugs = []
        for agency in self.agencies_json.get('agencies', []):
            agency_name = agency.get('name', 'Unknown Agency')
            slug = agency.get('slug', 'Unknown Slug')
            # Add the slug to the list of slugs
            if slug not in self.slugs:
                self.slugs.append(slug)
            print(f"Processing agency: {agency_name} with SLUG: {slug}")
            # loop through all the titles, chapters, and parts for each agency
            for cfr_references in agency.get('cfr_references', []):
                title = cfr_references.get('title', 'Unknown Title')
                chapter = cfr_references.get('chapter', None)
                part = cfr_references.get('part', None)
                # fetch the version for each title chapter for this SLUG
                try:
                    self.refresh_title_and_chapter_versions(str(title), str(chapter))   
                except Exception as e:
                    print(f"Error fetching versions for title {title} chapter {chapter}: {e}")
                # if not exception, titleXchapterY_versions.json file should be created
            pass
        return "Versions fetched successfully."    

   

<h2>Asynch Testing & Unit Tests</h2>

In [4]:
# Asynch request testing

class Asynch_Request_Test:
    """
    A class to test asynchronous requests to the eCFR API.
    """

    def __init__(self, base_url):
        self.base_url = base_url
        self.api_wrapper = eCFR_API_Wrapper(base_url)

    async def fetch_data_asynch(self, title, chapter):
        """
        Fetch data asynchronously from the eCFR API.
        """
        url = title_and_chapter_versions_url + str(title) + ".json?chapter=" + str(chapter)
        async with aio.ClientSession() as session:
            async with session.get(url) as response:
                version = await response.json()
                # and write it to a file
                myfilename = "title"+str(title)+"chapter"+str(chapter)+"_versions.json"
                with open(self.data_path + 'versions/'+myfilename, 'w') as myversionfile:
                    json.dump(version, myversionfile)
        return myfilename
    
    def get_all_versions(self):
        """
        Get all missing versions of a title and chapter from the eCFR API.
        """
        echk = eCFR_HouseKeeping(data_path)
        agencies = None
        # get all the agency data   
        agencies = echk.load_agency_data()
        # blank the slug array
        slugs = []
        for agency in agencies.get('agencies', []):
            agency_name = agency.get('name', 'Unknown Agency')
            slug = agency.get('slug', 'Unknown Slug')
            # Add the slug to the list of slugs
            if slug not in slugs:
                slugs.append(slug)
            print(f"Processing agency: {agency_name} with SLUG: {slug}")
            # loop through all the titles, chapters, and parts for each agency
            for cfr_references in agency.get('cfr_references', []):
                title = cfr_references.get('title', 'Unknown Title')
                chapter = cfr_references.get('chapter', None)
                part = cfr_references.get('part', None)
                version = None
                # if we already have the version for this title and chapter, populate the version object
                try:
                    # try to load the version
                    version = echk.load_title_and_chapter_versions(str(title), str(chapter))              
                except: 
                    print(f"No existing version found for title {title} chapter {chapter}, fetching new version.")
                if version:
                    print(f"Existing version found for title {title} chapter {chapter}: {version}")
                else:
                    # else fetch the version for each title chapter for this SLUG
                    try:
                    # this is the synch call
                    # version= echk.refresh_title_and_chapter_versions(str(title), str(chapter))  
                    # this is the asynch call 
                        myfile = self.fetch_data_asynch(str(title), str(chapter))   
                        print(f"Version data for title {title} chapter {chapter} fetched asynch and written to {myfile}")
                    except Exception as e:
                        print(f"Error fetching versions for title {title} chapter {chapter}: {e}")
                # if not exception, titleXchapterY_versions.json file should be created
            pass
       
        return None
    

  

In [ ]:
import unittest

class TestECFRHouseKeeping(unittest.TestCase):
    def setUp(self):
        self.ecfr_hk = eCFR_HouseKeeping(data_path)
        self.asynch_test = Asynch_Request_Test(base_url)

    def test_fetch_agencies_data(self):
        agencies = self.ecfr_hk.fetch_agencies_data()
        self.assertIsNotNone(agencies)

    def test_fetch_titles_data(self):
        titles = self.ecfr_hk.fetch_titles_data()
        self.assertIsNotNone(titles)

    def test_refresh_agencies(self):
        result = self.ecfr_hk.refresh_agencies()
        self.assertIn("Agency data available", result)

    def test_refresh_titles(self):
        result = self.ecfr_hk.refresh_titles()
        self.assertIn("Titles data available", result)
    
    def test_refresh_title_and_chapter_versions(self):
        result = self.ecfr_hk.refresh_title_and_chapter_versions("1", "I")
        self.assertIn("Version data available", result)

    def test_load_agency_data(self):
        agencies = self.ecfr_hk.load_agency_data()
        self.assertIsNotNone(agencies)

    def test_load_title_data(self):
        titles = self.ecfr_hk.load_title_data()
        self.assertIsNotNone(titles)    

'''
    def test_asynch_request_test(self):
        result = self.asynch_test.get_all_versions()
        self.assertIsNone(result)   
    

    def test_load_title_and_chapter_versions(self):
        versions = self.ecfr_hk.load_title_and_chapter_versions("1", "I")
        self.assertIsNotNone(versions)

    def test_refresh_all_versions_byslug(self):
        result = self.ecfr_hk.refresh_all_versions_byslug()
        self.assertIn("Versions fetched successfully", result)  
'''

unittest.main(argv=[''], verbosity=2, exit=False)


test_asynch_request_test (__main__.TestECFRHouseKeeping.test_asynch_request_test) ... 

Processing agency: Administrative Conference of the United States with SLUG: administrative-conference-of-the-united-states
Processing agency: Construction Industry Collective Bargaining Commission with SLUG: construction-industry-collective-bargaining-commission


<h2>Put classes and constants above<h2>
<h3>Put implementation below</h3>

In [ ]:

# initialize variables
titles_json= {'titles':[]}  #the titles_json dictionary to contain data
statusmsg = "initialized"  #use this to track what's going on 
lastbestdate = "1776-07-04"  #when was the eCFR last updated?
today = dt.date.today() #what day is it?

# test successful initialization
assert len(titles_json)>0
assert len(statusmsg)>0
assert str(today)>lastbestdate

print("Welcome to the MRWeCFR with base_url of "+base_url)

# Psuedocode
# 1. Let's see if we have a list of recent titles (less than 30 days old) in our titles.json file
# 2. If we do, let's see if we have the full and amendments of similar age, else, try and refresh titles.json 
# 3. If we are good with source data, skip to analysis, else, try and refresh the eCFR json details 
# 4. Analysis - let's load some basic semantic checks (wordcount, reading level) and some change trackers (count of changes, frequency over time)
# To do the analysis, we're going to have to download 2 things - the full text (for wordcount and reading level) and the amendments (for change tracking).


#let's check the titles.json file for freshness
# ecfr_housekeeping = eCFR_HouseKeeping(data_path)

# implementation to follow






Welcome to the MRWeCFR with base_url of https://www.ecfr.gov/api/
